In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda
from sklearn.preprocessing import MinMaxScaler
import skfuzzy as fuzz
import skfuzzy.control as ctrl

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('air_quality_health_impact_data.csv')

In [5]:
df.head()


,RecordID,AQI,PM10,PM2_5,NO2,SO2,O3,Temperature,Humidity,WindSpeed,RespiratoryCases,CardiovascularCases,HospitalAdmissions,HealthImpactScore,HealthImpactClass
0,1,187.270059,295.853039,13.038560,6.639263,66.161150,54.624280,5.150335,84.424344,6.137755,7,5,1,97.244041,0.0
1,2,475.357153,246.254703,9.984497,16.318326,90.499523,169.621728,1.543378,46.851415,4.521422,10,2,0,100.000000,0.0
2,3,365.996971,84.443191,23.111340,96.317811,17.875850,9.006794,1.169483,17.806977,11.157384,13,3,0,100.000000,0.0
3,4,299.329242,21.020609,14.273403,81.234403,48.323616,93.161033,21.925276,99.473373,15.302500,8,8,1,100.000000,0.0
4,5,78.009320,16.987667,152.111623,121.235461,90.866167,241.795138,9.217517,24.906837,14.534733,9,0,1,95.182643,0.0


In [6]:
df.describe


<bound method NDFrame.describe of       RecordID         AQI        PM10       PM2_5         NO2        SO2  \
0            1  187.270059  295.853039   13.038560    6.639263  66.161150   
1            2  475.357153  246.254703    9.984497   16.318326  90.499523   
2            3  365.996971   84.443191   23.111340   96.317811  17.875850   
3            4  299.329242   21.020609   14.273403   81.234403  48.323616   
4            5   78.009320   16.987667  152.111623  121.235461  90.866167   
...        ...         ...         ...         ...         ...        ...   
5806      5807  171.112762   11.246387  197.984628  158.643107  17.743678   
5807      5808  490.691667  275.340762   55.774170  132.336871  29.334724   
5808      5809  314.841798   41.892699  184.708551   82.105823  68.334578   
5809      5810  208.080473  165.533785  199.177255  100.796385  87.586488   
5810      5811   83.269257   82.216262  119.968244  193.440691  79.849177   

              O3  Temperature   Humidity 

In [7]:
df.describe()

,RecordID,AQI,PM10,PM2_5,NO2,SO2,O3,Temperature,Humidity,WindSpeed,RespiratoryCases,CardiovascularCases,HospitalAdmissions,HealthImpactScore,HealthImpactClass
count,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000,5811.000000
mean,2906.000000,248.438476,148.654997,100.223714,102.293445,49.456838,149.312431,14.975500,54.776853,9.989177,9.974187,4.988986,2.001033,93.785223,0.281191
std,1677.635538,144.777638,85.698500,58.096612,57.713175,28.530329,86.534240,14.483067,26.020786,5.776950,3.129234,2.216791,1.398794,13.318904,0.714075
min,1.000000,0.005817,0.015848,0.031549,0.009625,0.011023,0.001661,-9.990998,10.001506,0.002094,1.000000,0.000000,0.000000,22.448488,0.000000
25%,1453.500000,122.951293,75.374954,49.435171,53.538538,24.887264,73.999665,2.481500,31.995262,4.952343,8.000000,3.000000,1.000000,98.203057,0.000000
50%,2906.000000,249.127841,147.634997,100.506337,102.987736,49.530165,149.559871,14.942428,54.543904,10.051742,10.000000,5.000000,2.000000,100.000000,0.000000
75%,4358.500000,373.630668,222.436759,151.340260,151.658516,73.346617,223.380126,27.465374,77.641639,14.971840,12.000000,6.000000,3.000000,100.000000,0.000000
max,5811.000000,499.858837,299.901962,199.984965,199.980195,99.969561,299.936812,39.963434,99.997493,19.999139,23.000000,14.000000,12.000000,100.000000,4.000000


In [8]:
df.columns

Index(['RecordID', 'AQI', 'PM10', 'PM2_5', 'NO2', 'SO2', 'O3', 'Temperature',
       'Humidity', 'WindSpeed', 'RespiratoryCases', 'CardiovascularCases',
       'HospitalAdmissions', 'HealthImpactScore', 'HealthImpactClass'],
      dtype='object')

In [9]:
df.shape

(5811, 15)

In [10]:
selected_columns = ['AQI', 'PM10', 'PM2_5', 'NO2', 'SO2', 'O3']
new_df = df[selected_columns]

In [11]:
new_df.head()

,AQI,PM10,PM2_5,NO2,SO2,O3
0,187.270059,295.853039,13.038560,6.639263,66.161150,54.624280
1,475.357153,246.254703,9.984497,16.318326,90.499523,169.621728
2,365.996971,84.443191,23.111340,96.317811,17.875850,9.006794
3,299.329242,21.020609,14.273403,81.234403,48.323616,93.161033
4,78.009320,16.987667,152.111623,121.235461,90.866167,241.795138


In [13]:
new_df.shape

(5811, 6)

In [14]:
type(new_df)

pandas.core.frame.DataFrame

In [15]:
health_columns = ['RespiratoryCases', 'CardiovascularCases', 'HospitalAdmissions'] 


In [16]:
# Extract features and labels
X_pollution = new_df.values
X_health = df[health_columns].values
y_score = df['HealthImpactScore'].values
y_binary = df['HealthImpactClass'].values

In [19]:
X_pollution

array([[187.27005942, 295.85303919,  13.03856044,   6.63926301,
         66.16114965,  54.62427998],
       [475.3571532 , 246.25470278,   9.98449713,  16.31832608,
         90.4995226 , 169.62172753],
       [365.99697091,  84.44319074,  23.11133977,  96.31781102,
         17.87585035,   9.0067936 ],
       ...,
       [314.84179763,  41.89269906, 184.70855139,  82.10582279,
         68.33457751, 105.56850348],
       [208.0804732 , 165.53378512, 199.17725515, 100.79638514,
         87.58648826, 166.46953697],
       [ 83.26925675,  82.21626223, 119.96824423, 193.44069117,
         79.84917706,  28.68188524]])

## Pre processing the Data

In [20]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [21]:
# Scale the features
pollution_scaler = StandardScaler()
health_scaler = MinMaxScaler(feature_range=(0, 1))

X_pollution_scaled = pollution_scaler.fit_transform(X_pollution)
X_health_scaled = health_scaler.fit_transform(X_health)


In [40]:
X_pollution_scaled


array([[-0.42253543,  1.71777463, -1.5008217 , -1.65754896,  0.58554352,
        -1.09432172],
       [ 1.56749487,  1.13897098, -1.55339493, -1.48982477,  1.43868712,
         0.23471685],
       [ 0.81206333, -0.7493402 , -1.32742698, -0.10354912, -1.10702213,
        -1.6215282 ],
       ...,
       [ 0.45869678, -1.24589677,  1.45433797, -0.34982237,  0.66172963,
        -0.50555354],
       [-0.27878253,  0.19697242,  1.70340497, -0.0259419 ,  1.33657526,
         0.19828663],
       [-1.14094577, -0.77532806,  0.33988608,  1.57945025,  1.06535591,
        -1.39414093]])

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
# Split the data into training and test sets
X_pollution_train, X_pollution_test, X_health_train, X_health_test, y_binary_train, y_binary_test, y_score_train, y_score_test = train_test_split(
    X_pollution_scaled, X_health_scaled, y_binary, y_score, test_size=0.2, random_state=42
)

In [24]:
X_pollution_train.shape

(4648, 6)

## Building the Neural Network

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Concatenate

In [29]:
# feature processing
# retrieving dimensions for pollution and health input
# Set the feature dimensions
pollution_feature_dim = X_pollution_train.shape[1]
health_feature_dim = X_health_train.shape[1]


In [30]:
# Pollution Inputs
pollution_inputs_ann = Input(shape=(pollution_feature_dim,), name='pollution_inputs_ann')

# Health Impact Inputs
health_inputs_ann = Input(shape=(health_feature_dim,), name='health_inputs_ann')

In [31]:
# Feature Processing
x_pollution_ann = Dense(64, activation='relu')(pollution_inputs_ann)
x_pollution_ann = Dropout(0.3)(x_pollution_ann)

x_health_ann = Dense(32, activation='relu')(health_inputs_ann)
x_health_ann = Dropout(0.2)(x_health_ann)


In [ ]:
def _build_multi_output_model(self):
        """
        Build neural network with multiple outputs
        """
        # Pollution Inputs
        pollution_inputs = Input(shape=(self.pollution_feature_dim,), name='pollution_inputs')
        
        # Health Impact Inputs
        health_inputs = Input(shape=(self.health_feature_dim,), name='health_inputs')
        
        # Feature Processing
        x_pollution = Dense(64, activation='relu')(pollution_inputs)
        x_pollution = Dropout(0.3)(x_pollution)
        
        x_health = Dense(32, activation='relu')(health_inputs)
        x_health = Dropout(0.2)(x_health)
        
        # Concatenate Features
        combined = Concatenate()([x_pollution, x_health])
        
        # Advanced Hidden Layers
        x = Dense(64, activation='relu')(combined)
        x = Dropout(0.3)(x)
        x = Dense(32, activation='relu')(x)
        x = Dropout(0.2)(x)
        
        # Multiple Outputs
        # Binary Classification Output
        binary_classification = Dense(1, activation='sigmoid', name='binary_health_class')(x)
        
        # Continuous Health Impact Score Regression Output
        health_impact_score = Dense(1, activation='linear', name='health_impact_score')(x)
        
        # Uncertainty Estimation
        uncertainty = Lambda(
            lambda tensor: tf.math.reduce_std(tensor, axis=-1), 
            name='uncertainty'
        )(x)
        
        # Create Model
        model = Model(
            inputs=[pollution_inputs, health_inputs], 
            outputs=[binary_classification, health_impact_score, uncertainty]
        )
        
        # Compile Model with Multiple Losses
        model.compile(
            optimizer='adam',
            loss={
                'binary_health_class': 'binary_crossentropy',
                'health_impact_score': 'mean_squared_error',
                'uncertainty': 'mean_squared_error'
            },
            loss_weights={
                'binary_health_class': 1.0,
                'health_impact_score': 0.5,
                'uncertainty': 0.1
            },
            metrics={
                'binary_health_class': ['accuracy']
            }
        )
        
        return model